# Requirements Files

In [ ]:
%%writefile SETTINGS.json
{
    "DATA_BASE_DIR": "../input/",
    "MAX_SEQ_LENGTH": 1536,
    "USE_FP16": true,
    "NUM_CORES": 16,
    "BATCH_SIZE": 4,
    "GRAD_ACCUM_STEPS": 2,
    "NUM_EPOCHS": 5,
    "LR": 2e-5
}

In [ ]:
%%writefile requirements.txt
torch==1.7.0
transformers==4.17.0
tensorboard==2.6.0
pandas==1.4.1
numpy==1.21.2
scipy==1.6.3

# Training

In [ ]:
%%writefile trainer.py
import os
import json
import pickle
import argparse
import numpy as np
import pandas as pd
import transformers
from functools import partial
import multiprocessing as mp
from scipy.special import softmax
from torch.utils.data import Dataset
from transformers import (AutoModelForTokenClassification,
                          DataCollatorForTokenClassification, 
                          TrainingArguments, 
                          Trainer,
                          TrainerCallback)

ap = argparse.ArgumentParser()
ap.add_argument("--fold", type=int)
ap.add_argument("--pretrained_model", type=str)
cargs = ap.parse_args()

if ("deberta-v3" in cargs.pretrained_model)|("deberta-v2" in cargs.pretrained_model):
    from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast as AutoTokenizer
else:
    from transformers import AutoTokenizer

cfg = json.load(open('SETTINGS.json', 'r'))

DATA_BASE_DIR = cfg["DATA_BASE_DIR"]
LR = cfg["LR"]
NUM_EPOCHS = cfg["NUM_EPOCHS"]
NUM_CORES = cfg["NUM_CORES"]
BATCH_SIZE = cfg["BATCH_SIZE"]
USE_FP16 = cfg["USE_FP16"]
GRAD_ACCUM_STEPS = cfg["GRAD_ACCUM_STEPS"]
MAX_SEQ_LENGTH = cfg["MAX_SEQ_LENGTH"]
SPLIT_NUM = cargs.fold
PRETRAINED_MODEL = cargs.pretrained_model

TRAIN_CSV = os.path.join(DATA_BASE_DIR, 'feedback-prize-2021/train.csv')
TRAIN_DIR = os.path.join(DATA_BASE_DIR, 'feedback-prize-2021/train/')

MIN_TOKENS = {
    "Lead": 6,
    "Position": 3,
    "Evidence": 20,
    "Claim": 1,
    "Concluding Statement": 3,
    "Counterclaim": 7,
    "Rebuttal": 6
}

train_df = pd.read_csv(TRAIN_CSV)
train_df['discourse_id'] = train_df['discourse_id'].astype('long').astype('str')
train_df['discourse_start'] = train_df['discourse_start'].astype('int')
train_df['discourse_end'] = train_df['discourse_end'].astype('int')

folds = pickle.load(open(os.path.join(DATA_BASE_DIR, 'feedbackgroupshufflesplit1337/groupshufflesplit_1337.p'), 'rb'))

ner_labels = ['O']
for curr_label in train_df['discourse_type'].unique():
    ner_labels.append('B-' + curr_label)
    ner_labels.append('I-' + curr_label)
ner_labels = dict((x,i) for i,x in enumerate(ner_labels))

inverted_ner_labels = dict((v,k) for k,v in ner_labels.items())
inverted_ner_labels[-100] = 'Special Token'


def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]

def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])
    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

def generate_token_to_word_mapping(txt, offset):
    # GET WORD POSITIONS IN CHARS
    w = []
    blank = True
    for i in range(len(txt)):
        if not txt[i].isspace() and blank==True:
            w.append(i)
            blank=False
        elif txt[i].isspace():
            blank=True
    w.append(1e6)

    # MAPPING FROM TOKENS TO WORDS
    word_map = -1 * np.ones(len(offset),dtype='int32')
    w_i = 0
    for i in range(len(offset)):
        if offset[i][1]==0: continue
        while offset[i][0]>=w[w_i+1]: w_i += 1
        word_map[i] = int(w_i)
        
    return word_map

class TextOverlapFBetaScore:
    def __init__(self, test_df, test_dataset):
        self.test_df = test_df
        self.test_dataset = test_dataset

    def __call__(self, raw_predictions):
        predictions, _ = raw_predictions
        soft_predictions = softmax(predictions, -1)
        soft_predictions = np.max(soft_predictions, axis=-1)
        predictions = np.argmax(predictions, axis=-1)

        all_preds = []
        # Clumsy gathering of predictions at word lvl - only populate with 1st subword pred
        for curr_sample_id in range(len(self.test_dataset)):
            sample_preds = predictions[curr_sample_id]
            sample_offset = self.test_dataset.get_offset(curr_sample_id)
            sample_txt = ner_valid_rows[curr_sample_id][1]
            sample_word_map = generate_token_to_word_mapping(sample_txt, sample_offset)

            word_preds = [''] * (max(sample_word_map) + 1)
            word_probs = dict()
            for i, curr_word_id in enumerate(sample_word_map):
                if curr_word_id != -1 and word_preds[curr_word_id] == '': # only use 1st subword
                    word_preds[curr_word_id] = inverted_ner_labels[sample_preds[i]]
                    word_probs[curr_word_id] = soft_predictions[curr_sample_id, i]

            # Dict to hold Lead, Position, Concluding Statement
            let_one_dict = dict() # K = Type, V = (Prob of start token, start, end)

            # If we see tokens I-X, I-Y, I-X in a sequence -> change I-Y to I-X
            for j in range(1, len(word_preds) - 1):
                pred_trio = [word_preds[k] for k in [j - 1, j, j + 1]]
                splitted_trio = [x.split('-')[0] for x in pred_trio]
                if all([x == 'I' for x in splitted_trio]) and pred_trio[0] == pred_trio[2] and pred_trio[0] != pred_trio[1]:
                    word_preds[j] = word_preds[j-1]

            j = 0 # start of candidate discourse
            while j < len(word_preds): 
                cls = word_preds[j] 
                cls_splitted = cls.split('-')[-1]
                end = j + 1 # try to extend discourse as far as possible

                if word_probs[j] > 0.63: 
                    # Must match suffix i.e., I- to I- only; no B- to I-
                    while end < len(word_preds) and (word_preds[end].split('-')[-1] == cls_splitted if cls_splitted in ['Lead', 'Position', 'Concluding Statement'] else word_preds[end] == f'I-{cls_splitted}'):
                        end += 1
                    # if we're here, end is not the same pred as start
                    if cls != 'O' and end - j > MIN_TOKENS[cls_splitted]: # needs to be longer than class-specified min
                        if cls_splitted in ['Lead', 'Position', 'Concluding Statement']:
                            lpc_max_prob = max(word_probs[c] for c in range(j, end))
                            if cls_splitted in let_one_dict: # Already existing, check contiguous or higher prob
                                prev_prob, prev_start, prev_end = let_one_dict[cls_splitted]
                                if j - prev_end < 3: # If close enough, combine
                                    let_one_dict[cls_splitted] = (max(prev_prob, lpc_max_prob), prev_start, end)
                                elif lpc_max_prob > prev_prob: # Overwrite if current candidate is more likely
                                    let_one_dict[cls_splitted] = (lpc_max_prob, j, end)
                            else: # Add to it
                                let_one_dict[cls_splitted] = (lpc_max_prob, j, end)
                        else:
                            # Lookback and add preceding I- tokens
                            while j - 1 > 0 and word_preds[j-1] == cls:
                                j = j - 1
                            # Try to add the matching B- tag if immediately precedes the current I- sequence
                            if j - 1 > 0 and word_preds[j-1] == f'B-{cls_splitted}':
                                j = j - 1

                            all_preds.append((self.test_dataset.get_filename(curr_sample_id), 
                                              cls_splitted, 
                                              ' '.join(map(str, list(range(j, end+1))))))

                j = end 

            # Add the Lead, Position, Concluding Statement
            for k, v in let_one_dict.items():
                pred_start = v[1]
                # Lookback and add preceding I- tokens
                while pred_start - 1 > 0 and word_preds[pred_start-1] == f'I-{k}':
                    pred_start = pred_start -1
                # Try to add the matching B- tag if immediately precedes the current I- sequence
                if pred_start - 1 > 0 and word_preds[pred_start - 1] == f'B-{k}':
                    pred_start = pred_start - 1

                all_preds.append((self.test_dataset.get_filename(curr_sample_id), 
                                  k, 
                                  ' '.join(map(str, list(range(pred_start, v[2]))))))

        output_df = pd.DataFrame(all_preds)
        output_df.columns = ['id', 'class', 'predictionstring']

        f1s = []
        CLASSES = output_df['class'].unique()
        for c in CLASSES:
            pred_df = output_df.loc[output_df['class']==c].copy()
            gt_df = self.test_df.loc[self.test_df['discourse_type']==c].copy()
            f1 = score_feedback_comp(pred_df, gt_df)
            f1s.append(f1)

        return {"textoverlapfbeta": np.mean(f1s)}

class SaveBestModelCallback(TrainerCallback):
    def __init__(self):
        self.bestScore = 0

    def on_train_begin(self, args, state, control, **kwargs):
        assert args.evaluation_strategy != "no", "SaveBestModelCallback requires IntervalStrategy of steps or epoch"

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        metric_value = metrics.get("eval_textoverlapfbeta")
        if metric_value > self.bestScore:
            print(f"** TextOverlapFBeta score improved from {np.round(self.bestScore, 4)} to {np.round(metric_value, 4)} **")
            self.bestScore = metric_value
            control.should_save = True
        else:
            print(f"TextOverlapFBeta score {np.round(metric_value, 4)} (Prev. Best {np.round(self.bestScore, 4)}) ")

valid_df = train_df.iloc[folds[SPLIT_NUM][1]].reset_index(drop=True)
train_df = train_df.iloc[folds[SPLIT_NUM][0]].reset_index(drop=True)

train_files = train_df['id'].unique()
valid_files = valid_df['id'].unique()

# accepts file path, returns tuple of (file_ID, txt, NER labels)
def generate_NER_labels_for_file(input_filename, df):
    curr_id = input_filename.split('.')[0]
    with open(os.path.join(TRAIN_DIR, '{}.txt'.format(input_filename))) as f:
        curr_txt = f.read()

    # Set all token labels initially to non-label
    curr_labels = [ner_labels['O']] * len(curr_txt)
    # Iterate thru all labels associated w/ file and update labels
    curr_df = df[df['id']==curr_id]
    for curr_discourse in curr_df.itertuples():
        curr_discourse_label = curr_discourse.discourse_type 
        for curr_txt_idx in range(curr_discourse.discourse_start, 
                                  min(curr_discourse.discourse_end+1, len(curr_labels))):
            if curr_txt_idx == curr_discourse.discourse_start:
                iob_label = ner_labels['B-' + curr_discourse_label]
            else:
                iob_label = ner_labels['I-' + curr_discourse_label]
            curr_labels[curr_txt_idx] = iob_label
    assert curr_labels != [ner_labels['O']] * len(curr_txt)
    return curr_id, curr_txt, curr_labels

with mp.Pool(NUM_CORES) as p:
    ner_train_rows = p.map(partial(generate_NER_labels_for_file, df=train_df), train_files)

with mp.Pool(NUM_CORES) as p:
    ner_valid_rows = p.map(partial(generate_NER_labels_for_file, df=valid_df), valid_files)

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

# Check is rust-based fast tokenizer
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

def tokenize_and_align_labels(ner_raw_data):
    tokenized_inputs = tokenizer([x[1] for x in ner_raw_data], 
                                 max_length=MAX_SEQ_LENGTH,
                                 return_offsets_mapping=True,
                                 truncation=True)
    labels = []
    word_ids = []
    for i, char_label in enumerate([x[2] for x in ner_raw_data]):
        curr_word_ids = tokenized_inputs.word_ids(batch_index=i)
        curr_offset_mappings = tokenized_inputs['offset_mapping'][i]
        label_ids = []
        for j, word_idx in enumerate(curr_word_ids):
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            else:
                # Use label of 1st character of word
                # See offset that's 1 index after end of tokens for some reason
                char_idx = min(curr_offset_mappings[j][0], len(char_label)-1)
                label_ids.append(char_label[char_idx])
        word_ids.append(curr_word_ids)
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    tokenized_inputs['word_ids'] = word_ids
    tokenized_inputs['id'] = [x[0] for x in ner_raw_data]
    return tokenized_inputs

tokenized_all_train = tokenize_and_align_labels(ner_train_rows)
tokenized_all_valid = tokenize_and_align_labels(ner_valid_rows)

class NERDataset(Dataset):
    def __init__(self, input_dict):
        self.input_dict = input_dict
        
    def __getitem__(self, index):
        return {k:self.input_dict[k][index] for k in self.input_dict.keys() if k not in {'id', 'offset_mapping', 'word_ids'}}
    
    def get_filename(self, index):
        return self.input_dict['id'][index]
    
    def get_offset(self, index):
        return self.input_dict['offset_mapping'][index]
    
    def __len__(self):
        return len(self.input_dict['input_ids'])

train_dataset = NERDataset(tokenized_all_train)
valid_dataset = NERDataset(tokenized_all_valid)

model = AutoModelForTokenClassification.from_pretrained(PRETRAINED_MODEL, 
                                                        num_labels=len(ner_labels),
                                                        trust_remote_code=True)

model_name = PRETRAINED_MODEL.split('/')[-1]
args = TrainingArguments(f'{model_name}-{SPLIT_NUM}',
                         PRETRAINED_MODEL,
                         lr_scheduler_type='linear',
                         evaluation_strategy = 'steps',
                         eval_steps=500,
                         dataloader_num_workers=8,
                         learning_rate=LR,
                         log_level='warning',
                         fp16 = USE_FP16,
                         per_device_train_batch_size=BATCH_SIZE,
                         per_device_eval_batch_size=1,
                         gradient_accumulation_steps=GRAD_ACCUM_STEPS,
                         gradient_checkpointing=True,
                         num_train_epochs=NUM_EPOCHS,
                         save_strategy='no',
                         save_total_limit=1)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(model,
                  args=args,
                  train_dataset=train_dataset,
                  eval_dataset=valid_dataset,
                  compute_metrics=TextOverlapFBetaScore(test_df=valid_df, test_dataset=valid_dataset),
                  callbacks=[SaveBestModelCallback],
                  data_collator=data_collator,
                  tokenizer=tokenizer)

trainer.train()

In [ ]:
! python trainer.py --fold 0 --pretrained_model microsoft/deberta-large

In [ ]:
! python trainer.py --fold 1 --pretrained_model microsoft/deberta-large

In [ ]:
! python trainer.py --fold 2 --pretrained_model microsoft/deberta-large

In [ ]:
! python trainer.py --fold 3 --pretrained_model microsoft/deberta-large

In [ ]:
! python trainer.py --fold 0 --pretrained_model microsoft/deberta-xlarge

In [ ]:
! python trainer.py --fold 1 --pretrained_model microsoft/deberta-xlarge

In [ ]:
! python trainer.py --fold 2 --pretrained_model microsoft/deberta-xlarge

In [ ]:
! python trainer.py --fold 3 --pretrained_model microsoft/deberta-xlarge

In [ ]:
! python trainer.py --fold 4 --pretrained_model microsoft/deberta-xlarge

In [ ]:
! python trainer.py --fold 5 --pretrained_model microsoft/deberta-xlarge

# Inference Script with Post Process

In [ ]:
%%writefile generate_preds.py

import os
import argparse

ap = argparse.ArgumentParser()
ap.add_argument('--model_paths', nargs='+', required=True)
ap.add_argument("--save_name", type=str, required=True)
ap.add_argument("--max_len", type=int, required=True)
args = ap.parse_args()

        
import gc
import pickle
import numpy as np
import pandas as pd
import transformers
import multiprocessing as mp
from scipy.special import softmax
from torch.utils.data import Dataset
from transformers import (AutoModelForTokenClassification, 
                          AutoTokenizer, 
                          TrainingArguments, 
                          Trainer)

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

NUM_CORES = 16
BATCH_SIZE = 4
MAX_SEQ_LENGTH = args.max_len
PRETRAINED_MODEL_PATHS = args.model_paths
if "debertal_chris" in args.save_name:
    print('==> using -1 in offset mapping...')
    
AGG_FUNC = np.mean
print('==> using span token mean...')

TEST_DIR = '../input/feedback-prize-2021/test/'

MIN_TOKENS = {
    "Lead": 32,
    "Position": 5,
    "Evidence": 35,
    "Claim": 7,
    "Concluding Statement": 6,
    "Counterclaim": 6,
    "Rebuttal": 6
}

if "chris" not in args.save_name:
    ner_labels = {'O': 0,
                  'B-Lead': 1,
                  'I-Lead': 2,
                  'B-Position': 3,
                  'I-Position': 4,
                  'B-Evidence': 5,
                  'I-Evidence': 6,
                  'B-Claim': 7,
                  'I-Claim': 8,
                  'B-Concluding Statement': 9,
                  'I-Concluding Statement': 10,
                  'B-Counterclaim': 11,
                  'I-Counterclaim': 12,
                  'B-Rebuttal': 13,
                  'I-Rebuttal': 14}
else:
    print("==> Using Chris BIO")
    ner_labels = {'O': 14,
                  'B-Lead': 0,
                  'I-Lead': 1,
                  'B-Position': 2,
                  'I-Position': 3,
                  'B-Evidence': 4,
                  'I-Evidence': 5,
                  'B-Claim': 6,
                  'I-Claim': 7,
                  'B-Concluding Statement': 8,
                  'I-Concluding Statement': 9,
                  'B-Counterclaim': 10,
                  'I-Counterclaim': 11,
                  'B-Rebuttal': 12,
                  'I-Rebuttal': 13}


inverted_ner_labels = dict((v,k) for k,v in ner_labels.items())
inverted_ner_labels[-100] = 'Special Token'

test_files = os.listdir(TEST_DIR)

# accepts file path, returns tuple of (file_ID, txt split, NER labels)
def generate_text_for_file(input_filename):
    curr_id = input_filename.split('.')[0]
    with open(os.path.join(TEST_DIR, input_filename)) as f:
        curr_txt = f.read()

    return curr_id, curr_txt

with mp.Pool(NUM_CORES) as p:
    ner_test_rows = p.map(generate_text_for_file, test_files)
    

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_PATHS[0])

# Check is rust-based fast tokenizer
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

ner_test_rows = sorted(ner_test_rows, key=lambda x: len(tokenizer(x[1], max_length=MAX_SEQ_LENGTH, truncation=True)['input_ids']))

# tokenize and store word ids
def tokenize_with_word_ids(ner_raw_data):
    # ner_raw_data is shaped (num_examples, 3) where cols are (ID, words, word-level labels)
    tokenized_inputs = tokenizer([x[1] for x in ner_raw_data], 
                                 max_length=MAX_SEQ_LENGTH,
                                 return_offsets_mapping=True,
                                 truncation=True)
    
    tokenized_inputs['id'] = [x[0] for x in ner_raw_data]
    tokenized_inputs['offset_mapping'] = [tokenized_inputs['offset_mapping'][i] for i in range(len(ner_raw_data))]
    
    return tokenized_inputs

tokenized_all = tokenize_with_word_ids(ner_test_rows)

class NERDataset(Dataset):
    def __init__(self, input_dict):
        self.input_dict = input_dict
        
    def __getitem__(self, index):
        return {k:self.input_dict[k][index] for k in self.input_dict.keys() if k not in {'id', 'offset_mapping'}}
    
    def get_filename(self, index):
        return self.input_dict['id'][index]
    
    def get_offset(self, index):
        return self.input_dict['offset_mapping'][index]
    
    def __len__(self):
        return len(self.input_dict['input_ids'])

test_dataset = NERDataset(tokenized_all)

soft_predictions = None
hfargs = TrainingArguments(output_dir='None',
                         log_level='warning',
                         per_device_eval_batch_size=BATCH_SIZE)

for idx, curr_path in enumerate(PRETRAINED_MODEL_PATHS):

    
    model = AutoModelForTokenClassification.from_pretrained(curr_path, trust_remote_code=True)
    trainer = Trainer(model,
                      hfargs,
                      tokenizer=tokenizer)
    
    curr_preds, _, _ = trainer.predict(test_dataset)
    curr_preds = curr_preds.astype(np.float16)
    curr_preds = softmax(curr_preds, -1)

    if soft_predictions is not None:
        soft_predictions = soft_predictions + curr_preds
    else:
        soft_predictions = curr_preds
        
    del model, trainer, curr_preds
    gc.collect()

soft_predictions = soft_predictions / len(PRETRAINED_MODEL_PATHS)

soft_claim_predictions = soft_predictions[:, :, 8]

predictions = np.argmax(soft_predictions, axis=2)
soft_predictions = np.max(soft_predictions, axis=2)

def generate_token_to_word_mapping(txt, offset):
    # GET WORD POSITIONS IN CHARS
    w = []
    blank = True
    for i in range(len(txt)):
        if not txt[i].isspace() and blank==True:
            w.append(i)
            blank=False
        elif txt[i].isspace():
            blank=True
    w.append(1e6)

    # MAPPING FROM TOKENS TO WORDS
    word_map = -1 * np.ones(len(offset),dtype='int32')
    w_i = 0
    for i in range(len(offset)):
        if offset[i][1]==0: continue
        while offset[i][0]>=(w[w_i+1]-("debertal_chris" in args.save_name)-("v3" in args.save_name)\
                             -("v2" in args.save_name) ): w_i += 1
        word_map[i] = int(w_i)

    return word_map

all_preds = []

# Clumsy gathering of predictions at word lvl - only populate with 1st subword pred
for curr_sample_id in range(len(test_dataset)):
    curr_preds = []
    sample_preds = predictions[curr_sample_id]
    sample_offset = test_dataset.get_offset(curr_sample_id)
    sample_txt = ner_test_rows[curr_sample_id][1]
    sample_word_map = generate_token_to_word_mapping(sample_txt, sample_offset)

    word_preds = [''] * (max(sample_word_map) + 1)
    word_probs = dict(zip(range((max(sample_word_map) + 1)),[0]*(max(sample_word_map) + 1)))
    claim_probs = dict(zip(range((max(sample_word_map) + 1)),[0]*(max(sample_word_map) + 1)))

    for i, curr_word_id in enumerate(sample_word_map):
        if curr_word_id != -1:
            if word_preds[curr_word_id] == '': # only use 1st subword
                word_preds[curr_word_id] = inverted_ner_labels[sample_preds[i]]
                word_probs[curr_word_id] = soft_predictions[curr_sample_id, i]
                claim_probs[curr_word_id] = soft_claim_predictions[curr_sample_id, i]
            elif 'B-' in inverted_ner_labels[sample_preds[i]]:
                word_preds[curr_word_id] = inverted_ner_labels[sample_preds[i]]
                word_probs[curr_word_id] = soft_predictions[curr_sample_id, i]
                claim_probs[curr_word_id] = soft_claim_predictions[curr_sample_id, i]

    # Dict to hold Lead, Position, Concluding Statement
    let_one_dict = dict() # K = Type, V = (Prob of start token, start, end)

    # If we see tokens I-X, I-Y, I-X -> change I-Y to I-X
    for j in range(1, len(word_preds) - 1):
        pred_trio = [word_preds[k] for k in [j - 1, j, j + 1]]
        splitted_trio = [x.split('-')[0] for x in pred_trio]
        if all([x == 'I' for x in splitted_trio]) and pred_trio[0] == pred_trio[2] and pred_trio[0] != pred_trio[1]:
            word_preds[j] = word_preds[j-1]

    # B-X, ? (not B), I-X -> change ? to I-X
    for j in range(1, len(word_preds) - 1):
        if 'B-' in word_preds[j-1] and word_preds[j+1] == f"I-{word_preds[j-1].split('-')[-1]}" and word_preds[j] != word_preds[j+1] and 'B-' not in word_preds[j]:
            word_preds[j] = word_preds[j+1]

     # If we see tokens I-X, O, I-X, change center token to the same for stated discourse types
    for j in range(1, len(word_preds) - 1):
        if word_preds[j - 1] in ['I-Lead', 'I-Position', 'I-Concluding Statement'] and word_preds[j-1] == word_preds[j+1] and word_preds[j] == 'O':
            word_preds[j] = word_preds[j-1]

    j = 0 # start of candidate discourse
    while j < len(word_preds): 
        cls = word_preds[j] 
        cls_splitted = cls.split('-')[-1]
        end = j + 1 # try to extend discourse as far as possible

        if word_probs[j] > 0.54: 
            # Must match suffix i.e., I- to I- only; no B- to I-
            while end < len(word_preds) and (word_preds[end].split('-')[-1] == cls_splitted if cls_splitted in ['Lead', 'Position', 'Concluding Statement'] else word_preds[end] == f'I-{cls_splitted}'):
                end += 1
            # if we're here, end is not the same pred as start
            if cls != 'O' and (end - j > MIN_TOKENS[cls_splitted] or max(word_probs[l] for l in range(j, end)) > 0.73): # needs to be longer than class-specified min
                if cls_splitted in ['Lead', 'Position', 'Concluding Statement']:
                    lpc_max_prob = max(word_probs[c] for c in range(j, end))
                    if cls_splitted in let_one_dict: # Already existing, check contiguous or higher prob
                        prev_prob, prev_start, prev_end = let_one_dict[cls_splitted]
                        if cls_splitted in ['Lead', 'Concluding Statement'] and j - prev_end < 49: # If close enough, combine
                            let_one_dict[cls_splitted] = (max(prev_prob, lpc_max_prob), prev_start, end)
                            
                            # Delete other preds that lie inside the joined LC discourse
                            for l in range(len(curr_preds) - 1, 0, -1):
                                check_span = curr_preds[l][2]
                                check_start, check_end = int(check_span[0]), int(check_span[-1])
                                if check_start > prev_start and check_end < end:
                                    del curr_preds[l]
                            
                        elif lpc_max_prob > prev_prob: # Overwrite if current candidate is more likely
                            let_one_dict[cls_splitted] = (lpc_max_prob, j, end)
                    else: # Add to it
                        let_one_dict[cls_splitted] = (lpc_max_prob, j, end)
                else:
                    # Lookback and add preceding I- tokens
                    while j - 1 > 0 and word_preds[j-1] == cls:
                        j = j - 1
                    # Try to add the matching B- tag if immediately precedes the current I- sequence
                    if j - 1 > 0 and word_preds[j-1] == f'B-{cls_splitted}':
                        j = j - 1


                    #############################################################
                    # Run a bunch of adjustments to discourse predictions based on CV 
                    adj_start, adj_end = j, end + 1

                    # Run some heuristics against previous discourse
                    if len(curr_preds) > 0:
                        prev_span = list(map(int, curr_preds[-1][2].split()))
                        prev_start, prev_end = prev_span[0], prev_span[-1]

                        # Join adjacent rebuttals
                        if cls_splitted in 'Rebuttal':                        
                            if curr_preds[-1][1] == cls_splitted and adj_start - prev_end < 32:
                                del curr_preds[-1]
                                combined_list = prev_span + list(range(adj_start, adj_end))                                
                                curr_preds.append((test_dataset.get_filename(curr_sample_id), 
                                                   cls_splitted, 
                                                   ' '.join(map(str, combined_list)),
                                                   AGG_FUNC([word_probs[i] for i in combined_list if i in word_probs.keys()])))
                                j = end
                                continue
                                
                        elif cls_splitted in 'Counterclaim':                        
                            if curr_preds[-1][1] == cls_splitted and adj_start - prev_end < 24:
                                del curr_preds[-1]
                                combined_list = prev_span + list(range(adj_start, adj_end))                                
                                curr_preds.append((test_dataset.get_filename(curr_sample_id), 
                                                   cls_splitted, 
                                                   ' '.join(map(str, combined_list)),
                                                  AGG_FUNC([word_probs[i] for i in combined_list if i in word_probs.keys()])))
                                j = end
                                continue

                        elif cls_splitted in 'Evidence':                        
                            if curr_preds[-1][1] == cls_splitted and 8 < adj_start - prev_end < 25:
                                if max(claim_probs[l] for l in range(prev_end+1, adj_start)) > 0.35:
                                    claim_tokens = [str(l) for l in range(prev_end+1, adj_start) if claim_probs[l] > 0.15]
                                    if len(claim_tokens) > 2:
                                        curr_preds.append((test_dataset.get_filename(curr_sample_id), 
                                                           'Claim', 
                                                           ' '.join(claim_tokens),
                                                           AGG_FUNC([word_probs[int(i)] for i in claim_tokens if int(i) in word_probs.keys()])))
                        # If gap with discourse of same type, extend to it 
                        elif curr_preds[-1][1] == cls_splitted and adj_start - prev_end > 2:
                            adj_start -= 1

                    # Adjust discourse lengths if too long or short
                    if cls_splitted == 'Evidence':
                        if adj_end - adj_start < 45:
                            adj_start -= 9
                        else:
                            adj_end -= 1
                    elif cls_splitted == 'Claim':
                        if adj_end - adj_start > 24:
                            adj_end -= 1
                    elif cls_splitted == 'Counterclaim':
                        if adj_end - adj_start > 24:
                            adj_end -= 1
                        else:
                            adj_start -= 1
                            adj_end += 1
                    elif cls_splitted == 'Rebuttal':
                        if adj_end - adj_start > 32:
                            adj_end -= 1
                        else:
                            adj_start -= 1
                            adj_end += 1
                    adj_start = max(0, adj_start)
                    adj_end = min(len(word_preds) - 1, adj_end)
                    curr_preds.append((test_dataset.get_filename(curr_sample_id), 
                                       cls_splitted, 
                                       ' '.join(map(str, list(range(adj_start, adj_end)))),
                                       AGG_FUNC([word_probs[i] for i in range(adj_start, adj_end) if i in word_probs.keys()])))

        j = end 

    # Add the Lead, Position, Concluding Statement
    for k, v in let_one_dict.items():
        pred_start = v[1]
        pred_end = v[2]

        # Lookback and add preceding I- tokens
        while pred_start - 1 > 0 and word_preds[pred_start-1] == f'I-{k}':
            pred_start = pred_start - 1
        # Try to add the matching B- tag if immediately precedes the current I- sequence
        if pred_start - 1 > 0 and word_preds[pred_start - 1] == f'B-{k}':
            pred_start = pred_start - 1

        # Extend short Leads and Concluding Statements
        if k == 'Lead':
            if pred_end - pred_start < 33:
                pred_end = min(len(word_preds), pred_end + 5)
            else:
                pred_end -= 5
        elif k == 'Concluding Statement':
            if pred_end - pred_start < 23:
                pred_start = max(0, pred_start - 1)
                pred_end = min(len(word_preds), pred_end + 10)
        elif k == 'Position':
            if pred_end - pred_start < 18:
                pred_end = min(len(word_preds), pred_end + 3)

        pred_start = max(0, pred_start)
        if pred_end - pred_start > 6:
            curr_preds.append((test_dataset.get_filename(curr_sample_id), 
                               k, 
                               ' '.join(map(str, list(range(pred_start, pred_end)))),
                               AGG_FUNC([word_probs[i] for i in range(pred_start, pred_end) if i in word_probs.keys()])))

    all_preds.extend(curr_preds)

output_df = pd.DataFrame(all_preds)
output_df.columns = ['id', 'class', 'predictionstring', 'scores']
output_df.to_csv(f'{args.save_name}.csv', index=False)

# Weighted Box Fusion

In [ ]:
import warnings
import numpy as np

def prefilter_boxes(boxes, scores, labels, weights, thr):
    # Create dict with boxes stored by its label
    new_boxes = dict()

    for t in range(len(boxes)):

        if len(boxes[t]) != len(scores[t]):
            print('Error. Length of boxes arrays not equal to length of scores array: {} != {}'.format(len(boxes[t]), len(scores[t])))
            exit()

        for j in range(len(boxes[t])):
            score = scores[t][j]
            if score < thr:
                continue
            label = labels[t][j]
            box_part = boxes[t][j]

            x = float(box_part[0])
            y = float(box_part[1])

            # Box data checks
            if y < x:
                warnings.warn('Y < X value in box. Swap them.')
                x, y = y, x

            # [label, score, weight, model index, x, y]
            b = [label, float(score) * weights[t], weights[t], t, x, y]
            if label not in new_boxes:
                new_boxes[label] = []
            new_boxes[label].append(b)

    # Sort each list in dict by score and transform it to numpy array
    for k in new_boxes:
        current_boxes = np.array(new_boxes[k])
        new_boxes[k] = current_boxes[current_boxes[:, 1].argsort()[::-1]]

    return new_boxes


def get_weighted_box(boxes, conf_type='avg'):
    """
    Create weighted box for set of boxes
    :param boxes: set of boxes to fuse
    :param conf_type: type of confidence one of 'avg' or 'max'
    :return: weighted box (label, score, weight, model index, x, y)
    """

    box = np.zeros(6, dtype=np.float32)
    conf = 0
    conf_list = []
    w = 0
    for b in boxes:
        box[4:] += (b[1] * b[4:])
        conf += b[1]
        conf_list.append(b[1])
        w += b[2]
    box[0] = boxes[0][0]
    if conf_type == 'avg':
        box[1] = conf / len(boxes)
    elif conf_type == 'max':
        box[1] = np.array(conf_list).max()
    elif conf_type in ['box_and_model_avg', 'absent_model_aware_avg']:
        box[1] = conf / len(boxes)
    box[2] = w
    box[3] = -1 # model index field is retained for consistensy but is not used.
    box[4:] /= conf
    return box


def find_matching_box_quickly(boxes_list, new_box, match_iou):
    """ 
        Reimplementation of find_matching_box with numpy instead of loops. Gives significant speed up for larger arrays
        (~100x). This was previously the bottleneck since the function is called for every entry in the array.

        boxes_list: shape: (N, label, score, weight, model index, x, y)
        new_box: shape: (label, score, weight, model index, x, y)
    """
    def bb_iou_array(boxes, new_box):
        '''
        boxes: shape: (N, x, y)
        new_box: shape: (x, y)
        '''
        # bb interesection over union
        x_min = np.minimum(boxes[:, 0], new_box[0])
        x_max = np.maximum(boxes[:, 0], new_box[0])
        y_min = np.minimum(boxes[:, 1], new_box[1])+1
        y_max = np.maximum(boxes[:, 1], new_box[1])+1

        iou = np.maximum(0, (y_min-x_max)/(y_max-x_min))

        return iou

    if boxes_list.shape[0] == 0:
        return -1, match_iou

    # boxes = np.array(boxes_list)
    boxes = boxes_list

    ious = bb_iou_array(boxes[:, 4:], new_box[4:])

    ious[boxes[:, 0] != new_box[0]] = -1

    best_idx = np.argmax(ious)
    best_iou = ious[best_idx]

    if best_iou <= match_iou:
        best_iou = match_iou
        best_idx = -1

    return best_idx, best_iou


def weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=0.55, skip_box_thr=0.0, conf_type='avg', allows_overflow=False):
    '''
    :param boxes_list: list of boxes predictions from each model, each box is 2 numbers.
     It has 3 dimensions (models_number, model_preds, 2)
     Order of boxes: x, y.
    :param scores_list: list of scores for each model
    :param labels_list: list of labels for each model
    :param weights: list of weights for each model. Default: None, which means weight == 1 for each model
    :param iou_thr: IoU value for boxes to be a match
    :param skip_box_thr: exclude boxes with score lower than this variable
    :param conf_type: how to calculate confidence in weighted boxes. 'avg': average value, 'max': maximum value, 'box_and_model_avg': box and model wise hybrid weighted average, 'absent_model_aware_avg': weighted average that takes into account the absent model.
    :param allows_overflow: false if we want confidence score not exceed 1.0

    :return: boxes: boxes coordinates (Order of boxes: x, y).
    :return: scores: confidence scores
    :return: labels: boxes labels
    '''

    if weights is None:
        weights = np.ones(len(boxes_list))
    if len(weights) != len(boxes_list):
        print('Warning: incorrect number of weights {}. Must be: {}. Set weights equal to 1.'.format(len(weights), len(boxes_list)))
        weights = np.ones(len(boxes_list))
    weights = np.array(weights)

    if conf_type not in ['avg', 'max', 'box_and_model_avg', 'absent_model_aware_avg']:
        print('Unknown conf_type: {}. Must be "avg", "max" or "box_and_model_avg", or "absent_model_aware_avg"'.format(conf_type))
        exit()

    filtered_boxes = prefilter_boxes(boxes_list, scores_list, labels_list, weights, skip_box_thr)
    if len(filtered_boxes) == 0:
        return np.zeros((0, 2)), np.zeros((0,)), np.zeros((0,))

    overall_boxes = []
    for label in filtered_boxes:
        boxes = filtered_boxes[label]
        new_boxes = []
        weighted_boxes = np.empty((0,6)) ## [label, score, weight, model index, x, y]
        # Clusterize boxes
        for j in range(0, len(boxes)):
            index, best_iou = find_matching_box_quickly(weighted_boxes, boxes[j], iou_thr)

            if index != -1:
                new_boxes[index].append(boxes[j])
                weighted_boxes[index] = get_weighted_box(new_boxes[index], conf_type)
            else:
                new_boxes.append([boxes[j].copy()])
                weighted_boxes = np.vstack((weighted_boxes, boxes[j].copy()))

        # Rescale confidence based on number of models and boxes
        for i in range(len(new_boxes)):
            clustered_boxes = np.array(new_boxes[i])
            if conf_type == 'box_and_model_avg':
                # weighted average for boxes
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * len(clustered_boxes) / weighted_boxes[i, 2]
                # identify unique model index by model index column
                _, idx = np.unique(clustered_boxes[:, 3], return_index=True)
                # rescale by unique model weights
                weighted_boxes[i, 1] = weighted_boxes[i, 1] *  clustered_boxes[idx, 2].sum() / weights.sum()
            elif conf_type == 'absent_model_aware_avg':
                # get unique model index in the cluster
                models = np.unique(clustered_boxes[:, 3]).astype(int)
                # create a mask to get unused model weights
                mask = np.ones(len(weights), dtype=bool)
                mask[models] = False
                # absent model aware weighted average
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * len(clustered_boxes) / (weighted_boxes[i, 2] + weights[mask].sum())
            elif conf_type == 'max':
                weighted_boxes[i, 1] = weighted_boxes[i, 1] / weights.max()
            elif not allows_overflow:
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * min(len(weights), len(clustered_boxes)) / weights.sum()
            else:
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * len(clustered_boxes) / weights.sum()
        
        # REQUIRE BBOX TO BE PREDICTED BY AT LEAST 2 MODELS
        #for i in range(len(new_boxes)):
        #    clustered_boxes = np.array(new_boxes[i])
        #    if len(np.unique(clustered_boxes[:, 3])) > 1:
        #        overall_boxes.append(weighted_boxes[i])
                
        overall_boxes.append(weighted_boxes) # NOT NEEDED FOR "REQUIRE TWO MODELS" ABOVE
    overall_boxes = np.concatenate(overall_boxes, axis=0) # NOT NEEDED FOR "REQUIRE TWO MODELS" ABOVE
    #overall_boxes = np.array(overall_boxes) # NEEDED FOR "REQUIRE TWO MODELS" ABOVE
    overall_boxes = overall_boxes[overall_boxes[:, 1].argsort()[::-1]]
    boxes = overall_boxes[:, 4:]
    scores = overall_boxes[:, 1]
    labels = overall_boxes[:, 0]
    return boxes, scores, labels


# Folds

In [ ]:
!python generate_preds.py --model_paths ../input/deberta-large-100/fold0 \                                    
                            --save_name debertal_f0 --max_len 1536

In [ ]:
!python generate_preds.py --model_paths ../input/deberta-large-100/fold0 \
                                        ../input/deberta-large-100/fold1 \                                    
                            --save_name debertal_f1 --max_len 1536

In [ ]:
!python generate_preds.py --model_paths ../input/deberta-xlarge-1536/deberta-xlarge-v8004-f4/checkpoint-14000 \
                                                                    --save_name debertaxl-f4 --max_len 1536

In [ ]:
!python generate_preds.py --model_paths ../input/deberta-large-100/fold0 \
                                        ../input/deberta-large-100/fold1 \
                                        ../input/deberta-large-100/fold2 \
                            --save_name debertal --max_len 1536

!python generate_preds.py --model_paths ../input/deberta-xlarge-1536/deberta-xlarge-v8004-f4/checkpoint-14000 \
                                        ../input/deberta-xlarge-1536/deberta-xlarge-v4005-f5/checkpoint-13000 \
                            --save_name debertaxl --max_len 1536

In [ ]:
import pandas as pd
import numpy as np, os

debertaxl_csv = pd.read_csv("debertaxl.csv").dropna()
debertal_csv = pd.read_csv("debertal.csv").dropna()

# Ensemble Models with WBF

In [ ]:
TEST_DIR = '../input/feedback-prize-2021/test/'
test_files = os.listdir(TEST_DIR)
v_ids = [f.replace('.txt','') for f in test_files]

In [ ]:
import math

class_to_label = {
    'Claim': 0, 
    'Evidence': 1, 
    'Lead':2, 
    'Position':3, 
    'Concluding Statement':4,
    'Counterclaim':5, 
    'Rebuttal':6
}

# Threshold found from CV
label_to_threshold = {
    0 : 0.275, #Claim
    1 : 0.375, #Evidence
    2 : 0.325, #Lead
    3 : 0.325, #Position
    4 : 0.4, #Concluding Statement
    5 : 0.275, #Counterclaim
    6 : 0.275 #Rebuttal
}

label_to_class = {v:k for k, v in class_to_label.items()}

def preprocess_for_wbf(df_list):
    boxes_list=[]
    scores_list=[]
    labels_list=[]
    
    for df in df_list:
        scores_list.append(df['scores'].values.tolist())
        labels_list.append(df['class'].map(class_to_label).values.tolist())
        predictionstring = df.predictionstring.str.split().values
        df_box_list = []
        for bb in predictionstring:
            df_box_list.append([int(bb[0]), int(bb[-1])])
        boxes_list.append(df_box_list)
    return boxes_list, scores_list, labels_list

def postprocess_for_wbf(idx, boxes_list, scores_list, labels_list):
    preds = []
    for box, score, label in zip(boxes_list, scores_list, labels_list):
        if score > label_to_threshold[label]: 
            start = math.ceil(box[0])
            end = int(box[1])
            preds.append((idx, label_to_class[label], ' '.join([str(x) for x in range(start, end+1)])))
    return preds

def generate_wbf_for_id(i):
    df2 = debertal_csv[debertal_csv['id']==i]
    df4 = debertaxl_csv[debertaxl_csv['id']==i]
    
    boxes_list, scores_list, labels_list = preprocess_for_wbf([df2,df4])
    nboxes_list, nscores_list, nlabels_list = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=0.33, conf_type='avg')

    return postprocess_for_wbf(i, nboxes_list, nscores_list, nlabels_list)

In [ ]:
import multiprocessing as mp

with mp.Pool(2) as p:
    list_of_list = p.map(generate_wbf_for_id, v_ids)

preds = [x for sub_list in list_of_list for x in sub_list]

In [ ]:
sub = pd.DataFrame(preds)
sub.columns = ["id", "class", "predictionstring"]
sub.to_csv('submission.csv', index=False)

# Visualize Test Predictions


In [ ]:
from pathlib import Path
from spacy import displacy

test_path = Path('../input/feedback-prize-2021/test')

colors = {
            'Lead': '#8000ff',
            'Position': '#2b7ff6',
            'Evidence': '#2adddd',
            'Claim': '#80ffb4',
            'Concluding Statement': 'd4dd80',
            'Counterclaim': '#ff8042',
            'Rebuttal': '#ff0000',
            'Other': '#007f00',
         }

def get_test_text(ids):
    with open(test_path/f'{ids}.txt', 'r') as file: data = file.read()
    return data

def visualize(df):
    ids = df["id"].unique()
    for i in range(len(ids)):
        ents = []
        example = ids[i]
        curr_df = df[df["id"]==example]
        text = " ".join(get_test_text(example).split())
        splitted_text = text.split()
        for i, row in curr_df.iterrows():
            predictionstring = row['predictionstring']
            predictionstring = predictionstring.split()
            wstart = int(predictionstring[0])
            wend = int(predictionstring[-1])
            ents.append({
                             'start': len(" ".join(splitted_text[:wstart])), 
                             'end': len(" ".join(splitted_text[:wend+1])), 
                             'label': row['class']
                        })
        ents = sorted(ents, key = lambda i: i['start'])

        doc2 = {
            "text": text,
            "ents": ents,
            "title": example
        }

        options = {"ents": ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement', 'Counterclaim', 'Rebuttal'], "colors": colors}
        displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)

In [ ]:
if len(sub["id"].unique())==5:
    visualize(sub)